<a href="https://colab.research.google.com/github/bessabra/bessabra/blob/main/pro2pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
from datetime import datetime, timedelta

base = pd.read_csv('/content/drive/MyDrive/Progressão Professores/Versões v19.12/servEncontradosv19.12.csv')

In [6]:
#Mudando o tipo de veriavel para datetime
base['INATIVIDADE']           = pd.to_datetime(base['INATIVIDADE'], errors="coerce")
base['OBITO']                 = pd.to_datetime(base['OBITO'], errors="coerce")
base['DATA DO REQUERIMENTO']  = pd.to_datetime(base['DATA DO REQUERIMENTO'], errors="coerce")

#Subtrai 5 anos da Data Requerimento
base['DR5'] = base['DATA DO REQUERIMENTO'] - timedelta(days=1825)

In [7]:
#Requerimento antes da aposentadoria ou obito
base.loc[(base['DATA DO REQUERIMENTO'] < base['INATIVIDADE']) | (base['DATA DO REQUERIMENTO'] < base['OBITO']), 'DRstatus'] ='01 - ATIVO PERMANENTE ou 08 - ATIVO EM OUTRO ORGAO'
base.loc[(base['DATA DO REQUERIMENTO'] > base['INATIVIDADE']) | (base['DATA DO REQUERIMENTO'] > base['OBITO']), 'DRstatus'] ='02 – APOSENTADO ou 15 - INSTITUIDOR PENSAO;'
base.loc[(base['DR5'] < base['INATIVIDADE']) | (base['DR5'] < base['OBITO']), 'DR5status'] ='01 - ATIVO PERMANENTE ou 08 - ATIVO EM OUTRO ORGAO'
base.loc[(base['DR5'] > base['INATIVIDADE']) | (base['DR5'] > base['OBITO']), 'DR5status'] ='02 – APOSENTADO ou 15 - INSTITUIDOR PENSAO;'

#Situação após o filtro de intervalo de cobertura
base.loc[(base['SITUAÇÃO'] == '01 - ATIVO PERMANENTE'), 'REQUERIMENTO'] ='ANÁLISE'
base.loc[(base['DRstatus'] == '02 – APOSENTADO ou 15 - INSTITUIDOR PENSAO;') & (base['DR5status']         == '02 – APOSENTADO ou 15 - INSTITUIDOR PENSAO;'), 'REQUERIMENTO'] ='INDEFERIDO'
base.loc[(base['DRstatus'] == '02 – APOSENTADO ou 15 - INSTITUIDOR PENSAO;') & (base['DR5status']         == '01 - ATIVO PERMANENTE ou 08 - ATIVO EM OUTRO ORGAO'), 'REQUERIMENTO'] ='A DECIDIR'
base.loc[(base['DRstatus'] == '01 - ATIVO PERMANENTE ou 08 - ATIVO EM OUTRO ORGAO') & (base['DR5status']  == '01 - ATIVO PERMANENTE ou 08 - ATIVO EM OUTRO ORGAO'), 'REQUERIMENTO'] ='ANÁLISE'

base.to_excel('ResultadoPreliminar.xlsx')

In [8]:
baseAnalise   = base.loc[(base['REQUERIMENTO'] == 'ANÁLISE') | ((base['DRstatus'] == '01 - ATIVO PERMANENTE ou 08 - ATIVO EM OUTRO ORGAO') & (base['DR5status'] == '01 - ATIVO PERMANENTE ou 08 - ATIVO EM OUTRO ORGAO'))]
baseIdeferido = base.loc[(base['DRstatus']     == '02 – APOSENTADO ou 15 - INSTITUIDOR PENSAO;') & (base['DR5status'] == '02 – APOSENTADO ou 15 - INSTITUIDOR PENSAO;')]
baseADecidir  = base.loc[(base['DRstatus']     == '02 – APOSENTADO ou 15 - INSTITUIDOR PENSAO;') & (base['DR5status'] == '01 - ATIVO PERMANENTE ou 08 - ATIVO EM OUTRO ORGAO')]

baseAnalise.to_excel('baseAnalisev19.12.xlsx')
baseIdeferido.to_excel('baseIdeferidov19.12.xlsx')
baseADecidir.to_excel('baseADecidirv19.12.xlsx')